In [ ]:
#모델을 짤거임.

#https://stackoverflow.com/questions/48140989/keras-lstm-input-dimension-setting
#(batch_size, time_series,input_dim)

#Twitter Sentiment Analysis using combined LSTM-CNN Models

In [1]:
from keras.models import Input, Model, Sequential
from keras.layers import Dropout, Activation, Add,Bidirectional,LSTM,MaxPooling1D, Conv1D,BatchNormalization, Flatten, Dense,Embedding

C:\Users\ie-45\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
from gensim.models.word2vec import Word2Vec
import glob
from konlpy.tag import Twitter 
import re
import pandas as pd
import numpy as np

C:\Users\ie-45\Anaconda3\lib\site-packages\gensim\utils.py:1209: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [4]:
word_model = Word2Vec.load("./model/last_train_model.model")
train_data_dir = glob.glob("./train_data/*.csv")

In [77]:
pretrained_weights = word_model.wv.syn0
vocab_size, embedding_size = pretrained_weights.shape
max_sequence_length = 2000 #train 최대 길이에 맞추고 test는 그길이보다 길면 잘라줌


C:\Users\ie-45\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.wv.vectors instead).
  """Entry point for launching an IPython kernel.


In [104]:
#첫모델 아직 input 안됨
def model_creation(): 
    input_layer = Input(shape=(max_sequence_length,))
    embedding_layer = Embedding(vocab_size,embedding_size,
        weights=[word_model.wv.syn0],
        input_length=max_sequence_length,trainable=False)(input_layer) 
    
    #embedding_layer_2 = np.expand_dims(X, axis=2)
    c0 = Dropout(0.2)(embedding_layer)
    c1 = (Conv1D(64,kernel_size = 5, border_mode='same'))(c0)
    c2 = (BatchNormalization())(c1)
    c3 = (Activation('relu'))(c2)
    c4 = (MaxPooling1D(pool_size=(4)))(c3)
    c5 = (Dropout(0.25))(c4)

    c6 = (Conv1D(64,3, border_mode='same'))(c5)
    c7 = (BatchNormalization())(c6)
    c8 = (Activation('relu'))(c7)
    c9 = (MaxPooling1D(pool_size=(4)))(c8)
    c10 = (Dropout(0.25))(c9)

    left1 = Bidirectional(LSTM(output_dim=256, init='uniform', inner_init='uniform',
                   forget_bias_init='one', return_sequences=True, activation='tanh',
                   inner_activation='sigmoid', input_shape=(32, 32)))(c10)
    left2 = Bidirectional(LSTM(output_dim=256, init='uniform', inner_init='uniform',
                   forget_bias_init='one', return_sequences=True, activation='tanh',
                   inner_activation='sigmoid', input_shape=(32, 32)))(left1)

    right1 =Bidirectional(LSTM(output_dim=256, init='uniform', inner_init='uniform',
               forget_bias_init='one', return_sequences=True, activation='tanh',
               inner_activation='sigmoid', input_shape=(32, 32), go_backwards=True))(c10)
    right2 =Bidirectional(LSTM(output_dim=256, init='uniform', inner_init='uniform',
               forget_bias_init='one', return_sequences=True, activation='tanh',
               inner_activation='sigmoid', input_shape=(32, 32), go_backwards=True))(right1)
    
    
    merged = Add()([left2,right2])

    h1 = (Flatten())(merged)
    h2 = (Dense(512,activation='relu'))(h1)
    h3 = (Dense(256,activation='relu'))(h2)
    h4 = (Dense(128,activation='relu'))(h3)

    h5 = (Dense(2,activation='sigmoid'))(h4)


    model = Model(inputs = input_layer , outputs = output)
    model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
    return model

In [16]:
tagger = Twitter()
def tagging(one_row):
    hangle = re.compile('[^ ㄱ-ㅣ가-힣0-9]+')
    result = hangle.sub('',one_row)
    result = tagger.pos(result,norm=True,stem = True)
    return result

def data_preprocessing(one_row):
    data = tagging(one_row)
    r = []
    results = []
    jung_bok = ""
    for (word, pumsa) in data:
        if not pumsa in ["Josa", "Eomi", "Punctuation",'Number'] and jung_bok !=word:
            r.append(word)
            jung_bok = word
    results.append((" ".join(r)).strip())  
    return results[0]

def data_making(input_data):
    data = pd.read_csv(input_data,encoding='euc-kr').iloc[:,1:]
    data['times'] =data['times'].str[:10]
    data['times'] = data['times'].apply(lambda x : x.replace('.',"-"))
    data['str'] = data['titles'].str[:]+data['articles']
    del data['titles']
    del data['articles']
    data['str'] = data['str'].apply(data_preprocessing)
    return data
def dataframe_preprocessing(data,k,d):
    dataframe = data_making(data)
    data = dataframe['str'][k].split()
    data = word_to_vec(data)
    result = np.zeros((2000,200))
    for i in range(len(data)):
        result[-(i+1)] = data[i]
    label = np.array(dataframe[d][k]).reshape(-1,1)
    return result,label
    
def word_to_vec(data):
    word_vec_list = []
    for txt in data:
        try:
            vec = word_model.wv[txt]
            word_vec_list.append(vec)
        except:
            pass
            
    return word_vec_list

In [171]:
train_data_dir[0]

'./train_data\\2017.06.01_2017.12.31_hmscience.csv'

In [360]:
#train_data
train_one_row,train_one_label = dataframe_preprocessing(train_data_dir[0],0,'d1')

In [264]:
train_two_row, train_two_label = dataframe_preprocessing(train_data_dir[0],1,'d1')

In [210]:
len(train_one_row)

446

In [241]:
def model_creation(): 
    input_layer = Input(shape=(1,200))
    #embedding_layer = Embedding(vocab_size,embedding_size,
    #    weights=[word_model.wv.syn0],
    #    input_length=max_sequence_length,trainable=False)(input_layer) 
    
    
    left1 = Bidirectional(LSTM(input_shape  = (1,2000),output_dim=256, init='uniform', inner_init='uniform',
                   forget_bias_init='one', return_sequences=True, activation='tanh',
                   inner_activation='sigmoid'))(input_layer)
    left2 = Bidirectional(LSTM(output_dim=256, init='uniform', inner_init='uniform',
                   forget_bias_init='one', return_sequences=True, activation='tanh',
                   inner_activation='sigmoid'))(left1)

    right1 =Bidirectional(LSTM(input_shape  = (1,2000), output_dim=256, init='uniform', inner_init='uniform',
               forget_bias_init='one', return_sequences=True, activation='tanh',
               inner_activation='sigmoid', go_backwards=True))(input_layer)
    right2 =Bidirectional(LSTM(output_dim=256, init='uniform', inner_init='uniform',
               forget_bias_init='one', return_sequences=True, activation='tanh',
               inner_activation='sigmoid', go_backwards=True))(right1)
    #embedding_layer_2 = np.expand_dims(X, axis=2)
    merged = Add()([left2,right2])
    c0 = Dropout(0.2)(merged)
    c1 = (Conv1D(64,kernel_size = 3, border_mode='same'))(c0)
    c2 = (BatchNormalization())(c1)
    c3 = (Activation('relu'))(c2)
    c4 = (MaxPooling1D(pool_size=(4)))(c3)
    c5 = (Dropout(0.25))(c4)

    c6 = (Conv1D(64,3, border_mode='same'))(c5)
    c7 = (BatchNormalization())(c6)
    c8 = (Activation('relu'))(c7)
    c9 = (MaxPooling1D(pool_size=(4)))(c8)
    c10 = (Dropout(0.25))(c9)

    #h1 = (Flatten())(c10)
    h2 = (Dense(512,activation='relu'))(c10)
    h3 = (Dense(256,activation='relu'))(h2)
    h4 = (Dense(128,activation='relu'))(h3)

    output = (Dense(2,activation='sigmoid'))(h4)


    model = Model(inputs = input_layer , outputs = output)
    model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
    return model

In [343]:
def model_creation(): 
    input_layer = Input(shape=(446,))
    embedding_layer = Embedding(input_dim=word_model.wv.vectors.shape[0], \
                                output_dim=word_model.wv.vectors.shape[1],\
                                weights=[word_model.wv.vectors]\
                               , trainable = False)
    f1 = embedding_layer(input_layer)
    
    left = Bidirectional(LSTM(128,input_shape = (None,1,2000)))(f1)

    #output = (Dense(2,activation='sigmoid'))(left)


    model = Model(inputs = input_layer , outputs = left)
    model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
    return model

In [312]:
model = model_creation()

(446, 1, 200)

In [345]:
model.fit(train_one_row,[train_one_label])

ValueError: Error when checking input: expected input_46 to have shape (446,) but got array with shape (200,)

In [387]:
train_one_row = train_one_row.reshape(1,train_one_row.shape[0],train_one_row.shape[1])

In [369]:
train_one_row,train_one_label = dataframe_preprocessing(train_data_dir[0],0,'d1')
#train_one_row = np.array(train_one_row)
#train_one_row = train_one_row.reshape(1,train_one_row.shape[0],train_one_row.shape[1])

In [ ]:
# (batch_size, timestep, input_dim)

In [415]:
pretrained_weights = word_model.wv.syn0
vocab_size, embedding_size = pretrained_weights.shape
max_sequence_length = 2000 #train 최대 길이에 맞추고 test는 그길이보다 길면 잘라줌
embedding_size

C:\Users\ie-45\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.wv.vectors instead).
  """Entry point for launching an IPython kernel.


200

In [396]:
def model_creation():    
    #embedding_layer = Embedding(input_dim=word_model.wv.vectors.shape[0], \
    #                            output_dim=word_model.wv.vectors.shape[1],\
    #                            weights=[word_model.wv.vectors]\
    #                           , trainable = False)
    model=Sequential()
    model.add(LSTM(128,input_shape=(446,200)))
    model.add(Dense(1,activation='sigmoid'))
    model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
    return model

In [416]:
def model_creation():    
    #embedding_layer = Embedding(input_dim=word_model.wv.vectors.shape[0], \
    #                            output_dim=word_model.wv.vectors.shape[1],\
    #                            weights=[word_model.wv.vectors]\
    #                           , trainable = False)
    model=Sequential()
    model.add(Bidirectional(LSTM(input_shape  = (max_sequence_length,embedding_size),output_dim=256, init='uniform', inner_init='uniform',
                   forget_bias_init='one', return_sequences=True, activation='tanh',
                   inner_activation='sigmoid')))
    model.add(Bidirectional(LSTM(output_dim=256, init='uniform', inner_init='uniform',
                   forget_bias_init='one', activation='tanh',
                   inner_activation='sigmoid')))
    model.add(Dense(1,activation='sigmoid'))
    model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
    return model

In [412]:
train_one_row,train_one_label = dataframe_preprocessing(train_data_dir[0],0,'d1')

In [7]:
max_sequence_length= 2000
embedding_size= 200

In [8]:
def model_creation():    
    #embedding_layer = Embedding(input_dim=word_model.wv.vectors.shape[0], \
    #                            output_dim=word_model.wv.vectors.shape[1],\
    #                            weights=[word_model.wv.vectors]\
    #                           , trainable = False)
    model=Sequential()
    model.add(Bidirectional(LSTM(input_shape  = (max_sequence_length,embedding_size),output_dim=256, init='uniform', inner_init='uniform',
                   forget_bias_init='one', return_sequences=True, activation='tanh',
                   inner_activation='sigmoid')))
    model.add(Bidirectional(LSTM(output_dim=256, init='uniform', inner_init='uniform',
                   forget_bias_init='one', activation='tanh', return_sequences=True,
                   inner_activation='sigmoid')))
    model.add(Dropout(0.2))
    model.add(Conv1D(10,kernel_size=3,border_mode='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling1D(pool_size=(4)))
    model.add(Dropout(0.2))
    model.add(Conv1D(10,kernel_size=3,border_mode='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling1D(pool_size=(4)))
    model.add(Dropout(0.2))
    model.add(Flatten())
    model.add(Dense(1024))
    model.add(Dense(512))
    model.add(Dense(256))
    model.add(Dense(1,activation='sigmoid'))
    model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
    return model

In [9]:
model = model_creation()

C:\Users\ie-45\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(input_shape=(2000, 200..., return_sequences=True, activation="tanh", unit_forget_bias=True, units=256, kernel_initializer="uniform", recurrent_initializer="uniform", recurrent_activation="sigmoid")`
  if __name__ == '__main__':
C:\Users\ie-45\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(activation="tanh", return_sequences=True, unit_forget_bias=True, units=256, kernel_initializer="uniform", recurrent_initializer="uniform", recurrent_activation="sigmoid")`
  if sys.path[0] == '':
C:\Users\ie-45\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(10, kernel_size=3, padding="same")`
  
C:\Users\ie-45\Anaconda3\lib\site-packages\ipykernel_launcher.py:19: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(10

In [17]:
train_one_row,train_one_label = dataframe_preprocessing(train_data_dir[0],0,'d1')
train_one_row = train_one_row.reshape(1,train_one_row.shape[0],train_one_row.shape[1])

In [22]:
train_data_dir[1]

'./train_data\\2017.07.01_2017.12.31_daeungjaeyak.csv'

In [21]:
train_data_list = []
train_label_list = []
for i in range(len(train_data_dir)):
    train_one_csv = pd.read_csv(train_data_dir[i],encoding = 'euc-kr')
    for j in range(len(train_one_csv)):
        if len(train_data_list) == 16:
            model.fit(train_data_list,train_label_list)
            train_data_list =[]
            train_label_list = []
        else :
            print(i,j)
            train_data, train_label = dataframe_preprocessing(train_data_dir[i],j,'d1')
            train_data_list.append(train_data)
            train_label_list.append(train_label)

0 0
0 1
0 2
0 3
0 4
1 0


KeyError: 'd1'

In [436]:
train_one_row.shape

(1, 2000, 200)

In [18]:
model.fit(train_one_row,train_one_label)

Epoch 1/1
1/1 [==============================] - 18s 18s/step - loss: 0.0151 - acc: 1.0000


[array(0, dtype=int64)]